In [1]:
import environment
import agent
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
from time import sleep
import torch

train = True
render = False
sub = True
epoch = 2000 if train else 1
agent = agent.DQNAgent(state_size=54, action_size=23, train=True, state_dict=None)
# agent = agent.DQNAgent(state_size=139, action_size=23, train=True, state_dict="weight/save_1.pth")
avg_reward, avg_score = 0, 0
score_log = []

for i in tqdm(range(epoch)):
    env = environment.FactoryEnv()
    
    if sub:
        if i == epoch-1:
            submission = pd.read_csv("data/sample_submission.csv")
            actions = []
            process_n = []        

    for step in range(2184):       
        r = 0
        s = 0
        state = env.get_state()
        mask = agent.get_mask(env)
        action = agent.get_action(state,mask)
        
        if sub:
            if i == epoch-1:
                action_name = env.action_map[action]
                action_process = 0
                if 'PROCESS' in action_name:
                    mylist = action_name.split("_")
                    action_name = mylist[0]
                    action_process = float(mylist[1])                
                if 'CHANGE' in action_name:
                    action_name = "CHANGE_" + str(env.prev_change+1) + str(action-3)
                actions.append(action_name)
                process_n.append(action_process)
      
        if render:
            print('Step : ' + str(env.step_count))
            print('Action : ' + str(env.action_map[action]))
            print('일일 생산량 : ' + str(env.day_process_n))
            print('Score : ' + str(env.prev_score))
            print('')
            
        next_state, reward, done = env.step(action)

        if train:
            agent.append_sample(state, action, reward, next_state, done)
            agent.train_model()
            r += reward 
            s += env.prev_score

        if done:
            break

#     if not train:
#         aaa = list()
#         bbb = list()
#         for j in range(4):
#             env.J[j].save_graph(f"performance/performance{j}.png")
#             info = np.array(env.J[j].info)
#             t, w, o = info[:, 0], info[:, 1], info[:, 2]
#             aaa.append(sum(w)/t[-1])
#             bbb.append(sum(o)/t[-1])
#         print(sum(aaa)/len(aaa))
#         print(sum(bbb)/len(bbb))
    
    if train:
        avg_score = s if avg_score == 0 else avg_score * 0.9 + s * 0.1
        score_log.append(env.prev_score)
        if i % 100 == 99:
            torch.save(agent.model.state_dict(), f"weight/save_{i + 1}_{avg_score}.pth")    
            
            # 그래프 그리기
            fig = plt.figure()
            plt.scatter(range(len(score_log)), score_log, s=1)
            plt.savefig(f"graph/save_{i + 1}_{avg_score}.png")
            plt.clf()
    
    
    
        
    

  0%|▎                                                                             | 8/2000 [03:01<13:12:57, 23.88s/it]

KeyboardInterrupt: 

In [ ]:
if sub:
    submission.loc[:, "Event_A"] = actions
    submission.loc[:, "MOL_A"] = process_n
    submission.loc[:, "Event_B"] = actions
    submission.loc[:, "MOL_B"] = process_n

    PRTs = submission[["PRT_1", "PRT_2", "PRT_3", "PRT_4"]].values
    for i in (np.array(env.PRT) - np.array([0,0,554])):
        if i[2] >= 0:
            PRTs[i[2], i[0]] = i[1]
    PRTs = np.ceil(PRTs * 1.1)
    PRTs = PRTs.astype(int)
    submission.loc[:, "PRT_1":"PRT_4"] = PRTs
    submission.to_csv(f"submission/save_{epoch}.csv", index=False)